In [39]:
#!pip install ipython-sql

import pandas as pd
import numpy as np
import econtools.metrics as mt
#import statsmodels.api as sm

In [40]:
df = pd.read_stata("data/output_data/Ability_Clean_2.dta")

In [41]:
#tbc ; egen hhid_tag=tag(hhid)
df["treat_dum"] = 0
df.loc[df["treat"]=="Treatment","treat_dum"] = 1
df.loc[(df["d_ave"].isnull()==False) & (df["d_ave"]>0),"overconf_ave"] = 1
df.loc[(df["d_ave"].isnull()==False) & (df["d_ave"]<0),"underestimate_ave"] = 1

varlist = ["exp_fees", "exp_uniforms", "exp_text", "exp_exbks", "exp_bks", "exp_tutoring", "exp_backpack", "exp_bks", "exp_exbks", "exp_text", "tot_bks_text", "tot_bks_all", "any_bks_text", "any_bks_all", "any_bks_text_s5", "any_bks_all_s5", "any_tutoring", "any_bks_text_tut", "any_exbks"]
for var in varlist:
    df.loc[(df[var]<0), var] = np.nan 
    
df["any_taking_tutoring"] = df.groupby(["hhid"])["taking_tutoring"].transform(max) 
df["one"] =1

In [42]:
#Respondent characteristics  
panelA=["primary_resp_fem", "resp_ed_decision", "age_par1", "educ", "secondary", "lit", "farmer", "inc_resp_wk"]

# HH Characteristics
panelB=["tot_kids", "one_par", "educ_ave", "any_secondary"]

#Student info
panelC=["std", "age", "female", "attendance_sv", "tot_exp", "exp_fees", "exp_uniforms", "tot_exp_suppl", "any_suppl"]

#Child performance 
panelD=["ave", "math", "engl", "chich", "math_engl"]

#Beliefs 
panelE=["b_ave", "b_math", "b_engl", "b_chich", "b_math_engl", "sd_ave"]

#misperceptions
panelF=["dp_ave", "dp_math", "dp_engl", "dp_chich", "dp_math_engl", "dp_ave_12", "d_ave", "overconf_ave", "wrong_12"]

#Beliefs about complementarity
panelG=["complementary"]

#Panel
panel_list = [panelA, panelB, panelC, panelD, panelE, panelF, panelG]


In [43]:
#table1
#overconf_ave 만 손보면 됨 index 39
#index 명칭, improve efficiency later



columns = [("Full Sample", "Mean"), ("Full Sample", "SD"), ("Control", "Mean"), ("Treat", "Mean"), ("Treat - Control","Mean")
          , ("Treat - Control","SE"), ("Treat - Control", "p-val T=C")]
columns = pd.MultiIndex.from_tuples(columns)

d = {}
for x in range(len(panel_list)):
    tab_1 = pd.DataFrame(columns = columns, index =range(0, len(panel_list[x])))
    tab_1 = tab_1.rename_axis("Panel"+str(x+1))
    d["table{0}".format(x+1)] = tab_1


for t in range(len(panel_list)):
    tab_1 = d["table"+str(t+1)]
    i = 0
    for pan in panel_list[t]:
        mean = round(df[pan].mean(),2)
        tab_1.loc[i,columns[0]] = mean
        
        std = round(df[pan].std(),2)
        tab_1.loc[i,columns[1]] = std
        
        mean_c = round(df.loc[df["treat_dum"]==0, pan].mean(),2)
        tab_1.loc[i,columns[2]] = mean_c
        
        mean_t = round(df.loc[df["treat_dum"]==1, pan].mean(),2)
        tab_1.loc[i,columns[3]] = mean_t
        
        result = mt.reg(df, pan, "treat_dum", fe_name = "school_code", cluster="hhid")
        result.summary[["coeff","se","p>t"]]
        tab_1.loc[i, columns[4]] = round(result.summary.loc["treat_dum","coeff"],2)
        tab_1.loc[i, columns[5]] = round(result.summary.loc["treat_dum","se"],2)
        tab_1.loc[i, columns[6]] = round(result.summary.loc["treat_dum","p>t"],2)
        
        i= i+1
        

C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


In [46]:
d["table1"]

Full Sample           Control    Treat Treat - Control          \
              Mean       SD     Mean     Mean            Mean      SE   
Panel1                                                                  
0             0.77     0.42     0.77     0.76           -0.01    0.02   
1             0.92     0.27     0.91     0.92            0.01    0.01   
2            40.81    10.97    40.65    40.97            0.32    0.44   
3             4.44     3.57     4.42     4.45            0.04    0.13   
4             0.11     0.31     0.11     0.11            0.01    0.01   
5             0.67     0.47     0.67     0.68            0.01    0.02   
6             0.46      0.5     0.47     0.46           -0.01    0.02   
7          2125.95  4744.03  2050.97  2202.59          196.86  194.09   

                  
       p-val T=C  
Panel1            
0           0.37  
1           0.31  
2           0.47  
3           0.78  
4           0.62  
5           0.67  
6            0.7  
7           0.31

In [ ]:
'''# areg 함수화 하기
#areg function
def areg(data=None,y=None,x=None,absorb=None,cluster=None):
    #data = data.dropna(subset =[x])
    #print(
    #    len(data.loc[data[x].isna() ==True].index.to_list()),
    #    len(data.loc[data[x].isna() ==False].index.to_list())
    #)
    
    y = data[y]
    x = data[x]
    x = sm.add_constant(x)
    reg = sm.OLS(exog = x, endog = y)
    return reg.fit(cov_type='cluster',cov_kwds={'groups':data[cluster].values})


y = df["primary_resp_fem"]
x = df["treat_dum"]
x = sm.add_constant(x)
model = sm.OLS(exog = x, endog = y)
result = model.fit(cov_type='cluster', cov_kwds = {'groups':df['hhid'].values})
result.summary()

def areg(data=None,y=None,x=None,absorb=None,cluster=None): 
    y = data[y]
    x = data[x]
    x = sm.add_constant(x)
    reg = sm.OLS(exog = x, endog = y)
    result = reg.fit(cov_type='cluster',cov_kwds={'groups':data[cluster].values})
    return print(result.summary())

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

def areg(formula,data=None,absorb=None,cluster=None): 

    y,X = patsy.dmatrices(formula,data,return_type='dataframe')

    ybar = y.mean()
    y = y -  y.groupby(data[absorb]).transform('mean') + ybar

    Xbar = X.mean()
    X = X - X.groupby(data[absorb]).transform('mean') + Xbar

    reg = sm.OLS(y,X)
    # Account for df loss from FE transform
    reg.df_resid -= (data[absorb].nunique() - 1)

    return reg.fit(cov_type='cluster',cov_kwds={'groups':data[cluster].values})

reg0 = areg("educ~treat_dum", data = df, absorb = "school_code", cluster = "hhid")
reg0.summary()'''